# Text simplification 
using code provided by https://github.com/chaojiang06/wiki-auto
Paper link: https://arxiv.org/abs/2005.02324

You need tensorflow 2 to be installed to run the code below!

## Data loading

## Data Preprocessing

In [3]:
#preprocess test_data/raw test_data/tokenized test_data/binarized
def preprocess_data(raw_data_dir,tokenized_data_dir,binarized_data_dir):
    !rm -r $binarized_data_dir/*
    !rm -r $tokenized_data_dir/*

    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/test.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/test.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/test.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/test.tok.dst

    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/valid.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/valid.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/valid.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/valid.tok.dst


    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/train.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/train.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/train.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/train.tok.dst


    # Creates binarized fairseq dataset
    !python simplification/preprocess.py --workers 5 --source-lang src --target-lang dst --trainpref $tokenized_data_dir/train.tok --validpref $tokenized_data_dir/valid.tok --testpref $tokenized_data_dir/test.tok --destdir  $binarized_data_dir --padding-factor 1 --joined-dictionary --srcdict simplification/preprocess/vocab_count.txt


In [9]:
preprocess_data('simplification/test_data/raw','simplification/test_data/tokenized','simplification/test_data/binarized')

2022-06-16 16:49:36.291913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 16:49:36.291961: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-16 16:49:39.946393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 16:49:39.946439: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-16 16:49:43.514139: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

## Text simplification generation

In [7]:
def generate_simplified_text(binarized_data_dir,output_file,checkpoint_file, gpu_id="", split="test"):
    print("Generating using checkpoint" , {checkpoint_file})
    print()
    !echo $output_file'.aner'

    !export CUDA_VISIBLE_DEVICES=$gpu_id 
    !python simplification/generate.py $binarized_data_dir --path $checkpoint_file --batch-size 32  --beam 1 --nbest 1 --user-dir simplification/my_model/ --print-alignment --gen-subset $split > $output_file'.aner'

    !python simplification/postprocess/bpe.py  --out_anon $output_file'.aner' --denon $output_file --ignore_lines 5 --wp 1

    !rm $output_file'.aner'

In [10]:
generate_simplified_text('simplification/test_data/binarized','simplification/0001.txt','checkpoints/checkpoint_wiki_auto.pt',0,"test")

Generating using checkpoint {'checkpoints/checkpoint_wiki_auto.pt'}

simplification/0001.txt.aner
2022-06-16 16:50:47.774006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 16:50:47.774053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
